In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, matthews_corrcoef, roc_auc_score, precision_recall_curve, roc_curve, auc
from sklearn.model_selection import StratifiedKFold
import random
from collections import deque
import matplotlib.pyplot as plt
import itertools
from math import comb
import seaborn as sns

print("=== 10-FOLD CROSS-VALIDATION ENSEMBLE F1-SCORE DQN OPTIMIZATION WITH DIRECT VOTING ===")
print("Focus: 10-fold CV with proper data flow + Direct model-level voting ensemble")
print("Training: DQN optimizes on fold validation data")
print("Learning Curve: Uses independent validation/test split within each fold")
print("Final Evaluation: Uses external validation dataset")
print("CRITICAL UPDATE: Models are retrained on each fold's training data")

# Calculate and display the number of possible combinations
print("\n🔢 COMBINATORIAL ANALYSIS:")
total_models = 36
print(f"Total models available: {total_models}")

total_combinations = 2**36 - 1
print(f"Total possible ensemble combinations: {total_combinations:,}")
print(f"In scientific notation: {total_combinations:.2e}")

# 36-BIT MODEL SELECTION STRUCTURE
print("\n🔢 36-BIT MODEL SELECTION STRUCTURE:")
print("   Bits 0-11:  Group 1 Models (Models 1-12 on Dataset 1)")
print("   Bits 12-23: Group 2 Models (Models 1-12 on Dataset 2)")
print("   Bits 24-35: Group 3 Models (Models 1-12 on Dataset 3)")
print("   Structure: [Group1: 000000000000][Group2: 000000000000][Group3: 000000000000]")

# =============================================================================
# DATA PREPARATION FOR 10-FOLD CROSS-VALIDATION
# =============================================================================
def prepare_10fold_datasets(original_datasets):
    """
    Combine train and test data, create 10-fold splits
    Keep external data separate for final evaluation
    """
    combined_datasets = []
    external_datasets = []
    
    for group_idx in range(3):
        X_train, X_test, y_train, y_test, X_external, y_external = original_datasets[group_idx]
        
        # Combine train and test data
        X_combined = np.concatenate([X_train, X_test], axis=0)
        y_combined = np.concatenate([y_train, y_test], axis=0)
        
        combined_datasets.append((X_combined, y_combined))
        external_datasets.append((X_external, y_external))
        
        print(f"Group {group_idx + 1}: Combined {len(X_combined)} samples, External {len(X_external)} samples")
    
    return combined_datasets, external_datasets

# CRITICAL: Before running this code, you must define the following variables:
# 
# Models (36 total):
# model1, model1c1, model1c2, model1c3, model1c4,
# model2, model2c1, model2c2, model2c3, model2c4, model2c5, model2c6,
# model3, model3c1, model3c2, model3c3, model3c4,
# model4, model4c1, model4c2, model4c3, model4c4, model4c5, model4c6,
# model5, model5c1, model5c2, model5c3, model5c4,
# model6, model6c1, model6c2, model6c3, model6c4, model6c5, model6c6
#
# Datasets (3 groups with train/test/external splits):
# X_train_1, X_test_1, y_train_1, y_test_1, X_external_1, y_external_1
# X_train_2, X_test_2, y_train_2, y_test_2, X_external_2, y_external_2  
# X_train_3, X_test_3, y_train_3, y_test_3, X_external_3, y_external_3

# Your real models - using the exact names from your original code
models_group_1 = [
     model1, model1c1, model1c2, model1c3, model1c4,
     model2, model2c1, model2c2, model2c3, model2c4, model2c5, model2c6
]
models_group_2 = [
     model3, model3c1, model3c2, model3c3, model3c4,
     model4, model4c1, model4c2, model4c3, model4c4, model4c5, model4c6
]
models_group_3 = [
     model5, model5c1, model5c2, model5c3, model5c4,
     model6, model6c1, model6c2, model6c3, model6c4, model6c5, model6c6
]
original_model_templates = models_group_1 + models_group_2 + models_group_3

# Prepare datasets for 10-fold CV
original_datasets = [
     (X_train_1, X_test_1, y_train_1, y_test_1, X_external_1, y_external_1),  # Group 1
     (X_train_2, X_test_2, y_train_2, y_test_2, X_external_2, y_external_2),  # Group 2
     (X_train_3, X_test_3, y_train_3, y_test_3, X_external_3, y_external_3)   # Group 3
]

combined_datasets, external_datasets = prepare_10fold_datasets(original_datasets)

print(f"\n✅ 10-FOLD CV DATA FLOW:")
print(f"✓ Combined datasets: {len(combined_datasets)} groups")
print(f"✓ External datasets: {len(external_datasets)} groups")
print(f"✓ CV Strategy: 10-fold with proper data separation")
print(f"✓ Model Training: Models retrained on each fold's training data (70%)")
print(f"✓ DQN Training: Uses dedicated DQN validation data (10%)")
print(f"✓ Learning Curve: Uses independent validation/test split (10% + 10%)")
print(f"✓ Final Evaluation: Uses X_external (completely untouched)")
print(f"✓ Voting Ensemble: Each selected model gets one vote per sample")

# =============================================================================
# FOLD DATA PREPARATION FUNCTION WITH PROPER SEPARATION
# =============================================================================
def create_fold_datasets(combined_datasets, n_folds=10, random_state=42):
    """
    Create stratified k-fold splits with proper data separation
    Each fold: 90% train, 10% test
    Training portion split into: 70% model training, 10% DQN validation, 10% learning curve validation
    Test portion used for: learning curve test
    """
    fold_datasets = []
    
    # Create stratified k-fold splitter
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    
    # Use labels from first group (all groups have identical labels)
    X_combined_group1, y_combined_group1 = combined_datasets[0]
    
    # Generate fold indices
    fold_indices = list(skf.split(X_combined_group1, y_combined_group1))
    
    for fold_idx, (train_indices, test_indices) in enumerate(fold_indices):
        fold_data = {}
        
        for group_idx in range(3):
            X_combined, y_combined = combined_datasets[group_idx]
            
            # Split into training (90%) and test (10%) for this fold
            X_train_fold = X_combined[train_indices]
            y_train_fold = y_combined[train_indices]
            X_test_fold = X_combined[test_indices]
            y_test_fold = y_combined[test_indices]
            
            # Further split training data (90%) into three parts:
            # - Model Training: 70% of total data (77.78% of training portion)
            # - DQN Validation: 10% of total data (11.11% of training portion)
            # - Learning Curve Validation: 10% of total data (11.11% of training portion)
            
            total_train_size = len(X_train_fold)
            model_training_size = int(0.7778 * total_train_size)  # 70%/90% = 0.7778
            dqn_validation_size = int(0.1111 * total_train_size)   # 10%/90% = 0.1111
            # Remaining goes to learning curve validation
            
            # Split training data
            X_model_training = X_train_fold[:model_training_size]
            y_model_training = y_train_fold[:model_training_size]
            
            X_dqn_validation = X_train_fold[model_training_size:model_training_size + dqn_validation_size]
            y_dqn_validation = y_train_fold[model_training_size:model_training_size + dqn_validation_size]
            
            X_learning_curve_validation = X_train_fold[model_training_size + dqn_validation_size:]
            y_learning_curve_validation = y_train_fold[model_training_size + dqn_validation_size:]
            
            # Test data for learning curve
            X_learning_curve_test = X_test_fold
            y_learning_curve_test = y_test_fold
            
            fold_data[group_idx] = {
                'model_training': (X_model_training, y_model_training),
                'dqn_validation': (X_dqn_validation, y_dqn_validation),  # For DQN rewards ONLY
                'learning_curve_validation': (X_learning_curve_validation, y_learning_curve_validation),  # For learning curve validation
                'learning_curve_test': (X_learning_curve_test, y_learning_curve_test)  # For learning curve test
            }
        
        fold_datasets.append(fold_data)
    
    return fold_datasets

# =============================================================================
# MODEL RETRAINING FUNCTION FOR EACH FOLD (WITH PROGRESS DISPLAY)
# =============================================================================
def retrain_models_for_fold(original_model_templates, fold_datasets, fold_idx, epochs=50, batch_size=32):
    """
    Retrain all 36 models on the current fold's model_training data
    Returns list of 36 retrained models
    Shows epoch and F1 score during training
    """
    print(f"\n{'='*70}")
    print(f"FOLD {fold_idx + 1}/10: RETRAINING 36 MODELS")
    print(f"{'='*70}")
    
    retrained_models = []
    
    for model_idx, original_model in enumerate(original_model_templates):
        group_idx = model_idx // 12
        
        # Get fold-specific training data
        X_train, y_train = fold_datasets[fold_idx][group_idx]['model_training']
        
        # Reshape data based on group
        if group_idx == 0:
            X_train_reshaped = X_train.reshape(-1, 150, 1)
        elif group_idx == 1:
            X_train_reshaped = X_train.reshape(-1, 84, 1)
        else:
            X_train_reshaped = X_train.reshape(-1, 420, 1)
        
        # Clone the original model architecture
        try:
            new_model = clone_model(original_model)
            new_model.compile(
                optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy']
            )
        except Exception as e:
            try:
                new_model = tf.keras.models.model_from_json(original_model.to_json())
                new_model.compile(
                    optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy']
                )
            except:
                new_model = clone_model(original_model)
                new_model.compile(
                    optimizer=Adam(learning_rate=0.001),
                    loss='binary_crossentropy',
                    metrics=['accuracy']
                )
        
        # Train the model on fold-specific data with custom callback for display
        print(f"\nModel {model_idx + 1}/36 (Group {group_idx + 1}):", end=" ")
        
        history = new_model.fit(
        X_train_reshaped, 
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,
        validation_split=0.1,
        callbacks=[early_stop]  # Add this line
        )
        
        # Get final epoch metrics
        final_val_loss = history.history['val_loss'][-1]
        final_val_acc = history.history['val_accuracy'][-1]
        
        # Calculate F1 score on validation data
        val_size = int(0.1 * len(X_train_reshaped))
        X_val = X_train_reshaped[-val_size:]
        y_val = y_train[-val_size:]
        
        y_pred_prob = new_model.predict(X_val, verbose=0).flatten()
        y_pred = (y_pred_prob > 0.5).astype(int)
        val_f1 = f1_score(y_val, y_pred, average='binary', zero_division=0)
        
        print(f"Epochs={epochs}, Val_Loss={final_val_loss:.4f}, Val_Acc={final_val_acc:.4f}, Val_F1={val_f1:.4f}")
        
        retrained_models.append(new_model)
    
    print(f"\n{'='*70}")
    print(f"✅ All 36 models retrained for Fold {fold_idx + 1}")
    print(f"{'='*70}")
    
    return retrained_models

# =============================================================================
# VISUALIZATION FUNCTIONS (SINGLE AGGREGATED LEARNING CURVES)
# =============================================================================
def create_aggregated_ensemble_training_plots(all_fold_rewards, all_fold_best_rewards, all_fold_f1_scores, all_fold_sizes):
    """Create single aggregated training reward curve with confidence intervals"""

    # Find the minimum length across all folds to ensure consistent aggregation
    min_length = min(len(rewards) for rewards in all_fold_rewards)
    
    # Truncate all arrays to minimum length
    truncated_rewards = [rewards[:min_length] for rewards in all_fold_rewards]
    truncated_best_rewards = [best_rewards[:min_length] for best_rewards in all_fold_best_rewards]
    
    # Convert to numpy arrays for easier computation
    rewards_array = np.array(truncated_rewards)
    best_rewards_array = np.array(truncated_best_rewards)
    
    # Calculate mean and std across folds
    mean_rewards = np.mean(rewards_array, axis=0)
    std_rewards = np.std(rewards_array, axis=0)
    mean_best_rewards = np.mean(best_rewards_array, axis=0)
    
    episodes = range(min_length)
    
    plt.figure(figsize=(3.5, 2.5))
    
    # Plot episode rewards with confidence interval
    plt.plot(episodes, mean_rewards, alpha=0.3, color='lightblue', label='Episode Rewards')
    plt.fill_between(episodes, mean_rewards - std_rewards, mean_rewards + std_rewards, 
                     alpha=0.2, color='lightblue')
    
    # Plot best reward so far
    plt.plot(episodes, mean_best_rewards, color='red', linewidth=1, label='Best Reward So Far')
    
    # Moving average
    window_size = min(100, min_length // 2)
    if min_length >= window_size:
        moving_avg = np.convolve(mean_rewards, np.ones(window_size)/window_size, mode='valid')
        plt.plot(range(window_size-1, min_length), moving_avg, 'blue', linewidth=1, label='Moving Average')

    # Add convergence line
    if len(mean_best_rewards) > 50:
        final_reward = mean_best_rewards[-1]
        convergence_threshold = final_reward * 0.99

        convergence_episode = None
        for i in range(len(mean_best_rewards)-1, 0, -1):
            if mean_best_rewards[i] < convergence_threshold:
                convergence_episode = i + 1
                break

        if convergence_episode:
            plt.axvline(x=convergence_episode, color='green', linestyle=':', linewidth=2,
                        label=f'Convergence (~Ep {convergence_episode})')

    plt.legend(fontsize=7.5, loc='lower right')
    plt.xlabel('Episode', fontsize=9, fontweight='bold')
    plt.ylabel('Reward', fontsize=9, fontweight='bold')
    plt.title('Ensemble Reward Convergence', fontsize=9, fontweight='bold')
    
    plt.grid(True, alpha=0.3)
    plt.ylim(0.4, 1.0)
    plt.yticks(fontsize=8)
    plt.xticks(fontsize=8)

    plt.tight_layout()
    plt.savefig('C:/Users/lenovo/Desktop/MY Work/Paper 4/image/Revision/Layer1_Reward.pdf')
    plt.show()

    
def create_aggregated_dqn_learning_plots(all_fold_agents, all_fold_f1_scores):
    """Create single aggregated DQN learning curves"""

    # Aggregate loss histories
    all_loss_histories = [agent.loss_history for agent in all_fold_agents if len(agent.loss_history) > 0]
    all_epsilon_histories = [agent.epsilon_history for agent in all_fold_agents if len(agent.epsilon_history) > 0]
    all_q_value_histories = [agent.q_value_history for agent in all_fold_agents if len(agent.q_value_history) > 0]

    # Plot 1: Aggregated DQN Training Loss
    plt.figure(figsize=(3, 2.5))
    if len(all_loss_histories) > 0:
        min_loss_length = min(len(loss_hist) for loss_hist in all_loss_histories)
        if min_loss_length > 0:
            truncated_loss = [loss_hist[:min_loss_length] for loss_hist in all_loss_histories]
            loss_array = np.array(truncated_loss)
            mean_loss = np.mean(loss_array, axis=0)
            std_loss = np.std(loss_array, axis=0)
            
            steps = range(min_loss_length)
            plt.plot(steps, mean_loss, alpha=0.7, color='red', linewidth=1)
            plt.fill_between(steps, mean_loss - std_loss, mean_loss + std_loss, 
                           alpha=0.3, color='red')

            if min_loss_length >= 50:
                window_size = 50
                moving_avg = np.convolve(mean_loss, np.ones(window_size) / window_size, mode='valid')
                plt.plot(range(window_size - 1, min_loss_length), moving_avg,
                         'darkred', linewidth=1, label='Moving Average (50)')
                plt.legend(fontsize=7.5, loc='upper right')
            
            plt.xlabel('Training Step', fontsize=9, fontweight='bold')
            plt.ylabel('DQN Loss (MSE)', fontsize=9, fontweight='bold')
            plt.title('DQN Training Loss', fontsize=9, fontweight='bold')
            plt.grid(True, alpha=0.3)
            
            if np.max(mean_loss) > 100:
                plt.yscale('log')
    else:
        plt.text(0.5, 0.5, 'No Loss Data Available', ha='center', va='center',
                 transform=plt.gca().transAxes, fontsize=10)
        plt.title('DQN Training Loss', fontsize=12, fontweight='bold')
    
    plt.yticks(fontsize=8)
    plt.xticks(fontsize=8)
    plt.tight_layout()
    plt.savefig('C:/Users/lenovo/Desktop/MY Work/Paper 4/image/Revision/Layer1_DQN_Loss.pdf')
    plt.show()

    # Plot 2: Aggregated Epsilon Decay
    plt.figure(figsize=(3, 2.5))
    if len(all_epsilon_histories) > 0:
        min_epsilon_length = min(len(eps_hist) for eps_hist in all_epsilon_histories)
        if min_epsilon_length > 0:
            truncated_epsilon = [eps_hist[:min_epsilon_length] for eps_hist in all_epsilon_histories]
            epsilon_array = np.array(truncated_epsilon)
            mean_epsilon = np.mean(epsilon_array, axis=0)
            
            steps = range(min_epsilon_length)
            plt.plot(steps, mean_epsilon, alpha=0.8, color='blue', linewidth=1.2)
            plt.xlabel('Training Step', fontsize=9, fontweight='bold')
            plt.ylabel('Epsilon (Exploration Rate)', fontsize=9, fontweight='bold')
            plt.title('Epsilon Decay', fontsize=9, fontweight='bold')
            plt.grid(True, alpha=0.3)

            plt.axhline(y=0.5, color='orange', linestyle='--', alpha=0.5, label='ε = 0.5')
            plt.axhline(y=0.1, color='green', linestyle='--', alpha=0.5, label='ε = 0.1')
            plt.axhline(y=0.01, color='red', linestyle='--', alpha=0.5, label='ε = 0.01 (min)')
            plt.legend(fontsize=8)
    else:
        plt.text(0.5, 0.5, 'No Epsilon Data Available', ha='center', va='center',
                 transform=plt.gca().transAxes, fontsize=10)
        plt.title('Epsilon Decay', fontsize=12, fontweight='bold')
    
    plt.yticks(fontsize=8)
    plt.xticks(fontsize=8)
    plt.tight_layout()
    plt.savefig('C:/Users/lenovo/Desktop/MY Work/Paper 4/image/Revision/Layer1_Epsilon_decay.pdf')
    plt.show()

    # Plot 3: Aggregated Q-Value Learning
    plt.figure(figsize=(3, 2.5))
    if len(all_q_value_histories) > 0:
        min_q_length = min(len(q_hist) for q_hist in all_q_value_histories)
        if min_q_length > 0:
            truncated_q = [q_hist[:min_q_length] for q_hist in all_q_value_histories]
            q_array = np.array(truncated_q)
            mean_q = np.mean(q_array, axis=0)
            std_q = np.std(q_array, axis=0)
            
            episodes = range(min_q_length)
            plt.plot(episodes, mean_q, alpha=0.6, color='purple', label='Max Q-Value')
            plt.fill_between(episodes, mean_q - std_q, mean_q + std_q, 
                           alpha=0.3, color='purple')

            window_size = min(50, min_q_length // 2)
            if min_q_length >= window_size:
                q_moving_avg = np.convolve(mean_q, np.ones(window_size) / window_size, mode='valid')
                plt.plot(range(window_size - 1, min_q_length), q_moving_avg,
                         'black', linewidth=1, label='Max Q-Value (50)')

            plt.xlabel('Episode', fontsize=9, fontweight='bold')
            plt.ylabel('Q-Value', fontsize=9, fontweight='bold')
            plt.title('Q-Value Learning', fontsize=9, fontweight='bold')
            plt.legend(fontsize=7.5, loc='lower right')
            plt.grid(True, alpha=0.3)
    else:
        plt.text(0.5, 0.5, 'No Q-Value Data Available', ha='center', va='center',
                 transform=plt.gca().transAxes, fontsize=10)
        plt.title('Q-Value Learning', fontsize=12, fontweight='bold')
    
    plt.yticks(fontsize=8)
    plt.xticks(fontsize=8)
    plt.tight_layout()
    plt.savefig('C:/Users/lenovo/Desktop/MY Work/Paper 4/image/Revision/Layer1_Q_Value.pdf')
    plt.show()

def create_aggregated_f1_learning_curve(all_fold_validation_f1, all_fold_test_f1, all_evaluation_episodes):
    """Create single aggregated F1 learning curve showing validation vs test F1 scores"""
    
    # Find minimum length across all folds
    min_length = min(len(val_f1) for val_f1 in all_fold_validation_f1 if len(val_f1) > 0)
    
    if min_length == 0:
        print("No F1 data available for learning curve")
        return
    
    # Truncate and aggregate F1 scores
    truncated_val_f1 = [val_f1[:min_length] for val_f1 in all_fold_validation_f1 if len(val_f1) >= min_length]
    truncated_test_f1 = [test_f1[:min_length] for test_f1 in all_fold_test_f1 if len(test_f1) >= min_length]
    
    if len(truncated_val_f1) == 0 or len(truncated_test_f1) == 0:
        print("Insufficient F1 data for aggregation")
        return
    
    val_f1_array = np.array(truncated_val_f1)
    test_f1_array = np.array(truncated_test_f1)
    
    mean_val_f1 = np.mean(val_f1_array, axis=0)
    std_val_f1 = np.std(val_f1_array, axis=0)
    mean_test_f1 = np.mean(test_f1_array, axis=0)
    std_test_f1 = np.std(test_f1_array, axis=0)
    
    # Use episodes from first fold (should be consistent across folds)
    episodes = all_evaluation_episodes[0][:min_length]
    
    plt.figure(figsize=(3, 2.5))
    
    # Plot validation and test F1 scores with confidence intervals
    plt.plot(episodes, mean_val_f1, alpha=0.7, color='blue', linewidth=0.5, label='Ensemble Validation F1')
    plt.fill_between(episodes, mean_val_f1 - std_val_f1, mean_val_f1 + std_val_f1, 
                     alpha=0.3, color='blue')
    
    plt.plot(episodes, mean_test_f1, alpha=0.7, color='red', linewidth=0.5, label='Ensemble Test F1')
    plt.fill_between(episodes, mean_test_f1 - std_test_f1, mean_test_f1 + std_test_f1, 
                     alpha=0.3, color='red')
    
    # Legend with font size
    plt.legend(fontsize=7.5, loc='lower right')
    
    plt.xlabel('Episode', fontsize=9, fontweight='bold')
    plt.ylabel('F1 Score', fontsize=9, fontweight='bold')
    plt.title('Validation vs Test F1 Score Validation', fontsize=9, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    plt.ylim(0.0, 1.0)
    plt.yticks(fontsize=8)
    plt.xticks(fontsize=8)
    
    plt.tight_layout()
    plt.savefig('C:/Users/lenovo/Desktop/MY Work/Paper 4/image/Revision/Layer1_F1_validation.pdf')
    plt.show()
    
    # Analysis across all folds
    print(f"\nF1 LEARNING CURVE ANALYSIS:")
    if len(mean_val_f1) > 0 and len(mean_test_f1) > 0:
        final_validation_f1 = mean_val_f1[-1]
        final_test_f1 = mean_test_f1[-1]
        final_gap = abs(final_validation_f1 - final_test_f1)
        
        print(f"   Final Validation F1: {final_validation_f1:.4f}")
        print(f"   Final Test F1: {final_test_f1:.4f}")
        print(f"   Final Gap: {final_gap:.4f}")
        
        if final_gap < 0.05:
            print(f"   STATUS: No overfitting - Good generalization")
        elif final_gap < 0.10:
            print(f"   STATUS: Slight overfitting - Acceptable")
        else:
            print(f"   STATUS: Overfitting detected - Poor generalization")

# =============================================================================
# DIRECT MODEL-LEVEL VOTING ENSEMBLE FUNCTION
# =============================================================================
def evaluate_direct_voting_ensemble_on_external_data(evaluator, selected_indices, external_datasets):
    """
    Direct voting ensemble: Each selected model gets one vote per sample
    """
    if len(selected_indices) == 0:
        return None
    
    valid_selected = [i for i in selected_indices if i in evaluator.valid_models]
    if len(valid_selected) == 0:
        return None
    
    print(f"Direct voting ensemble with {len(valid_selected)} models: {valid_selected}")
    
    # Collect votes from all selected models
    all_model_votes = []
    all_model_probabilities = []
    unified_true_labels = None
    
    for model_idx in valid_selected:
        group_idx = model_idx // 12
        
        # Get the external dataset for this model's group
        X_external, y_external = external_datasets[group_idx]
        
        # Store true labels (same across all datasets)
        if unified_true_labels is None:
            unified_true_labels = y_external.copy()
        
        # Reshape external data based on the group's feature dimensions
        if group_idx == 0:
            X_external_reshaped = X_external.reshape(-1, 150, 1)
        elif group_idx == 1:
            X_external_reshaped = X_external.reshape(-1, 84, 1)
        else:
            X_external_reshaped = X_external.reshape(-1, 420, 1)
        
        try:
            model = evaluator.models[model_idx]
            
            # Get probability predictions
            pred_prob = model.predict(X_external_reshaped, verbose=0).flatten()
            
            # Convert to binary vote (0 or 1)
            model_vote = (pred_prob > 0.5).astype(int)
            
            all_model_votes.append(model_vote)
            all_model_probabilities.append(pred_prob)
            print(f"  ✓ Model {model_idx} (Group {group_idx + 1}): {len(model_vote)} votes collected")
            
        except Exception as e:
            print(f"  ✗ Model {model_idx} failed: {e}")
            continue
    
    if len(all_model_votes) == 0:
        print("ERROR: No valid model votes!")
        return None
    
    # Direct majority voting across all selected models
    all_votes = np.array(all_model_votes)
    all_probs = np.array(all_model_probabilities)
    
    # Count votes for each sample
    positive_votes = np.sum(all_votes, axis=0)
    total_models = len(all_model_votes)
    
    # Majority rule: if more than half vote positive, predict positive
    final_predictions = (positive_votes > total_models / 2).astype(int)
    
    # Average probabilities for ROC-AUC and PR curves
    ensemble_probabilities = np.mean(all_probs, axis=0)
    
    print(f"\nDirect Voting Results:")
    print(f"├─ Total models voting: {total_models}")
    print(f"├─ Samples: {len(final_predictions)}")
    print(f"├─ Vote distribution per sample: min={np.min(positive_votes)}, max={np.max(positive_votes)}")
    print(f"└─ Final predictions: {np.bincount(final_predictions)}")
    
    # Calculate performance metrics
    accuracy = accuracy_score(unified_true_labels, final_predictions)
    f1 = f1_score(unified_true_labels, final_predictions, average='binary', zero_division=0)
    
    # Create confusion matrix
    cm = confusion_matrix(unified_true_labels, final_predictions)
    
    # Enhanced visualization with larger figure size
    plt.figure(figsize=(2.5, 2.5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'10-Fold CV Direct Voting Ensemble CM', 
              fontweight='bold', fontsize=9)
    plt.xlabel('Predicted Label', fontweight='bold', fontsize=9)
    plt.ylabel('True Label', fontweight='bold', fontsize=9)
    plt.xticks([0.5, 1.5], ['Negative (0)', 'Positive (1)'], fontsize=8)
    plt.yticks([0.5, 1.5], ['Negative (0)', 'Positive (1)'], fontsize=8)
    plt.tight_layout()
    plt.show()
    
    # Create ROC-AUC curve
    plt.figure(figsize=(3, 2.5))
    fpr, tpr, roc_thresholds = roc_curve(unified_true_labels, ensemble_probabilities)
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], color='red', lw=1, linestyle='--', label='Random Classifier')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontweight='bold', fontsize=9)
    plt.ylabel('True Positive Rate', fontweight='bold', fontsize=9)
    plt.title('10-Fold CV ROC-AUC Curve (External Data)', fontweight='bold', fontsize=9)
    plt.legend(loc="lower right", fontsize=8)
    plt.grid(True, alpha=0.3)
    plt.xticks(fontsize=8)
    plt.yticks(fontsize=8)
    plt.tight_layout()
    plt.savefig('C:/Users/lenovo/Desktop/MY Work/Paper 4/image/Revision/Layer1_10Fold_ROC_AUC.pdf')
    plt.show()
    
    # Create Precision-Recall curve
    plt.figure(figsize=(3, 2.5))
    precision, recall, pr_thresholds = precision_recall_curve(unified_true_labels, ensemble_probabilities)
    pr_auc = auc(recall, precision)
    
    plt.plot(recall, precision, color='green', lw=2, label=f'PR Curve (AUC = {pr_auc:.3f})')
    
    # Calculate baseline (random classifier performance)
    positive_ratio = np.sum(unified_true_labels) / len(unified_true_labels)
    plt.axhline(y=positive_ratio, color='red', linestyle='--', lw=1, 
                label=f'Random Classifier (Baseline = {positive_ratio:.3f})')
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall', fontweight='bold', fontsize=9)
    plt.ylabel('Precision', fontweight='bold', fontsize=9)
    plt.title('10-Fold CV Precision-Recall Curve (External Data)', fontweight='bold', fontsize=9)
    plt.legend(loc="lower left", fontsize=8)
    plt.grid(True, alpha=0.3)
    plt.xticks(fontsize=8)
    plt.yticks(fontsize=8)
    plt.tight_layout()
    plt.savefig('C:/Users/lenovo/Desktop/MY Work/Paper 4/image/Revision/Layer1_10Fold_PR_Curve.pdf')
    plt.show()
    
    # Enhanced detailed analysis with additional error metrics
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        total_samples = tn + fp + fn + tp
        
        # Basic metrics
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        
        # Additional error analysis metrics
        total_errors = fp + fn
        error_rate = total_errors / total_samples if total_samples > 0 else 0
        false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
        false_negative_rate = fn / (fn + tp) if (fn + tp) > 0 else 0
        
        print(f"\n10-FOLD CV DIRECT VOTING ENSEMBLE ANALYSIS:")
        print(f"├─ True Negatives (TN):   {tn:4d}")
        print(f"├─ False Positives (FP):  {fp:4d}")
        print(f"├─ False Negatives (FN):  {fn:4d}")
        print(f"├─ True Positives (TP):   {tp:4d}")
        print(f"└─ Total Samples:         {total_samples}")
        
        print(f"\nPerformance Metrics:")
        print(f"├─ Accuracy:              {accuracy:.4f}")
        print(f"├─ Sensitivity (Recall):  {sensitivity:.4f}")
        print(f"├─ Specificity:           {specificity:.4f}")
        print(f"├─ Precision (PPV):       {precision:.4f}")
        print(f"├─ F1 Score:              {f1:.4f}")
        print(f"├─ ROC-AUC:               {roc_auc:.4f}")
        print(f"└─ PR-AUC:                {pr_auc:.4f}")
    
    return {
        'confusion_matrix': cm,
        'true_labels': unified_true_labels,
        'predictions': final_predictions,
        'probabilities': ensemble_probabilities,
        'vote_counts': positive_votes,
        'total_models': total_models,
        'accuracy': accuracy,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'selected_models': valid_selected
    }

# =============================================================================
# 10-FOLD ENSEMBLE EVALUATOR CLASS (UPDATED FOR RETRAINED MODELS)
# =============================================================================
class TenFoldEnsembleEvaluator:
    """Evaluates ensemble performance with 10-fold cross-validation"""
    
    def __init__(self, models, combined_datasets, fold_datasets):
        self.models = models  # This will be updated with retrained models for each fold
        self.combined_datasets = combined_datasets
        self.fold_datasets = fold_datasets
        self.valid_models = self._validate_models()
        
        print(f"✅ 10-Fold ensemble evaluator initialized with {len(self.valid_models)}/36 valid models")
    
    def update_models(self, new_models):
        """Update models with newly retrained models for current fold"""
        self.models = new_models
        self.valid_models = self._validate_models()
        
    def _validate_models(self):
        """Validate which models can make predictions"""
        valid_models = []
        
        for i, model in enumerate(self.models):
            try:
                group_idx = i // 12
                # Use first fold's model training data for validation
                X_train_fold = self.fold_datasets[0][group_idx]['model_training'][0]
                
                # Build model if not built
                if not hasattr(model, '_built') or not model._built:
                    try:
                        if group_idx == 0:
                            input_shape = (None, 150, 1)
                        elif group_idx == 1:
                            input_shape = (None, 84, 1)
                        else:
                            input_shape = (None, 420, 1)
                        
                        model.build(input_shape=input_shape)
                    except Exception:
                        try:
                            sample_size = min(1, len(X_train_fold))
                            if group_idx == 0:
                                test_sample = X_train_fold[:sample_size].reshape(-1, 150, 1)
                            elif group_idx == 1:
                                test_sample = X_train_fold[:sample_size].reshape(-1, 84, 1)
                            else:
                                test_sample = X_train_fold[:sample_size].reshape(-1, 420, 1)
                            
                            _ = model(test_sample, training=False)
                        except Exception:
                            continue
                
                # Test prediction
                try:
                    sample_size = min(3, len(X_train_fold))
                    
                    if group_idx == 0:
                        test_sample = X_train_fold[:sample_size].reshape(-1, 150, 1)
                    elif group_idx == 1:
                        test_sample = X_train_fold[:sample_size].reshape(-1, 84, 1)
                    else:
                        test_sample = X_train_fold[:sample_size].reshape(-1, 420, 1)
                    
                    test_sample = tf.constant(test_sample, dtype=tf.float32)
                    test_pred = model(test_sample, training=False)
                    
                    if test_pred is not None and len(test_pred) == sample_size:
                        valid_models.append(i)
                    
                except Exception:
                    try:
                        test_pred = model.predict(test_sample.numpy(), verbose=0)
                        if test_pred is not None and len(test_pred) == sample_size:
                            valid_models.append(i)
                    except Exception:
                        continue
                
            except Exception:
                continue
                
        return valid_models
    
    def evaluate_ensemble_for_dqn_training(self, selected_indices, fold_idx):
        """
        Evaluate ensemble for DQN reward calculation
        Uses DQN validation data - this is what DQN optimizes on
        """
        if len(selected_indices) == 0:
            return 0.0
        
        valid_selected = [i for i in selected_indices if i in self.valid_models]
        if len(valid_selected) == 0:
            return 0.0
        
        all_predictions = []
        all_true_labels = []
        
        for group_idx in range(3):
            group_models = [i for i in valid_selected if i // 12 == group_idx]
            if len(group_models) == 0:
                continue
                
            # Use DQN validation data for DQN training
            X_dqn_validation, y_dqn_validation = self.fold_datasets[fold_idx][group_idx]['dqn_validation']
            
            # Reshape data for this group
            if group_idx == 0:
                X_dqn_validation_reshaped = X_dqn_validation.reshape(-1, 150, 1)
            elif group_idx == 1:
                X_dqn_validation_reshaped = X_dqn_validation.reshape(-1, 84, 1)
            else:
                X_dqn_validation_reshaped = X_dqn_validation.reshape(-1, 420, 1)
            
            # Get predictions from all models in this group
            group_predictions = []
            for model_idx in group_models:
                try:
                    model = self.models[model_idx]
                    pred_prob = model.predict(X_dqn_validation_reshaped, verbose=0).flatten()
                    group_predictions.append(pred_prob)
                except Exception:
                    continue
            
            if len(group_predictions) > 0:
                # Average predictions within group (ensemble voting)
                ensemble_prob = np.mean(group_predictions, axis=0)
                ensemble_pred = (ensemble_prob > 0.5).astype(int)
                
                all_predictions.extend(ensemble_pred)
                all_true_labels.extend(y_dqn_validation)
        
        if len(all_predictions) == 0:
            return 0.0
        
        # Calculate F1 score for DQN reward
        try:
            f1 = f1_score(all_true_labels, all_predictions, average='binary', zero_division=0)
            return f1
        except Exception:
            return 0.0
    
    def evaluate_ensemble_for_learning_curve(self, selected_indices, fold_idx):
        """
        Evaluate ensemble for learning curve
        Uses independent learning curve validation/test split within fold
        """
        if len(selected_indices) == 0:
            return 0.0, 0.0
        
        valid_selected = [i for i in selected_indices if i in self.valid_models]
        if len(valid_selected) == 0:
            return 0.0, 0.0
        
        # Collect predictions from learning curve validation portion
        all_validation_predictions = []
        all_validation_labels = []
        
        # Collect predictions from learning curve test portion  
        all_test_predictions = []
        all_test_labels = []
        
        for group_idx in range(3):
            group_models = [i for i in valid_selected if i // 12 == group_idx]
            if len(group_models) == 0:
                continue
            
            # Use independent learning curve validation/test data within fold
            X_validation, y_validation = self.fold_datasets[fold_idx][group_idx]['learning_curve_validation']
            X_test, y_test = self.fold_datasets[fold_idx][group_idx]['learning_curve_test']
            
            # Reshape data for this group
            if group_idx == 0:
                X_validation_reshaped = X_validation.reshape(-1, 150, 1)
                X_test_reshaped = X_test.reshape(-1, 150, 1)
            elif group_idx == 1:
                X_validation_reshaped = X_validation.reshape(-1, 84, 1)
                X_test_reshaped = X_test.reshape(-1, 84, 1)
            else:
                X_validation_reshaped = X_validation.reshape(-1, 420, 1)
                X_test_reshaped = X_test.reshape(-1, 420, 1)
            
            # Get predictions from all models in this group
            group_validation_predictions = []
            group_test_predictions = []
            
            for model_idx in group_models:
                try:
                    model = self.models[model_idx]
                    
                    # Learning curve validation predictions
                    validation_pred_prob = model.predict(X_validation_reshaped, verbose=0).flatten()
                    group_validation_predictions.append(validation_pred_prob)
                    
                    # Learning curve test predictions
                    test_pred_prob = model.predict(X_test_reshaped, verbose=0).flatten()
                    group_test_predictions.append(test_pred_prob)
                    
                except Exception:
                    continue
            
            if len(group_validation_predictions) > 0:
                # Average predictions within group
                ensemble_validation_prob = np.mean(group_validation_predictions, axis=0)
                ensemble_validation_pred = (ensemble_validation_prob > 0.5).astype(int)
                
                ensemble_test_prob = np.mean(group_test_predictions, axis=0)
                ensemble_test_pred = (ensemble_test_prob > 0.5).astype(int)
                
                all_validation_predictions.extend(ensemble_validation_pred)
                all_validation_labels.extend(y_validation)
                
                all_test_predictions.extend(ensemble_test_pred)
                all_test_labels.extend(y_test)
        
        if len(all_validation_predictions) == 0 or len(all_test_predictions) == 0:
            return 0.0, 0.0
        
        # Calculate F1 scores for learning curve
        try:
            validation_f1 = f1_score(all_validation_labels, all_validation_predictions, average='binary', zero_division=0)
            test_f1 = f1_score(all_test_labels, all_test_predictions, average='binary', zero_division=0)
            return validation_f1, test_f1
        except Exception:
            return 0.0, 0.0

# =============================================================================
# DQN AGENT CLASS (UPDATED WITH STATE SIZE = 36)
# =============================================================================
class EnsembleDQNAgent:
    def __init__(self, state_size=36, action_size=36, learning_rate=0.003):
        self.state_size = state_size  # 36-bit model selection state
        self.action_size = action_size  # 36 actions (one per model)
        self.memory = deque(maxlen=10000)
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = learning_rate
        self.gamma = 0.95
        self.q_network = self._build_model()
        self.target_network = self._build_model()
        self.update_target_network()
        
        # Learning tracking variables
        self.loss_history = []
        self.epsilon_history = []
        self.q_value_history = []
        self.reward_history = []

    def _build_model(self):
        model = Sequential()
        model.add(Dense(64, input_dim=self.state_size, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(self.action_size, activation='sigmoid'))  # Sigmoid for binary decisions
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def update_target_network(self):
        self.target_network.set_weights(self.q_network.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state, valid_models_list):
        """
        State: 36-bit current selection
        Action: 36-bit new selection based on Q-values
        """
        q_values = self.q_network.predict(state.reshape(1, -1), verbose=0)[0]
        max_q_value = np.max(q_values)
        self.q_value_history.append(max_q_value)
        
        if np.random.rand() <= self.epsilon:
            # Random exploration: randomly select 3-20 models
            action = np.zeros(36)
            max_select = min(len(valid_models_list), 20)
            min_select = 3
            num_select = np.random.randint(min_select, max_select + 1)
            selected = np.random.choice(valid_models_list, num_select, replace=False)
            action[selected] = 1
            return action
        
        # Exploitation: Use Q-values to decide inclusion/exclusion
        action = np.zeros(36)
        
        # Sort valid models by their Q-values
        valid_q_pairs = [(i, q_values[i]) for i in valid_models_list]
        valid_q_pairs.sort(key=lambda x: x[1], reverse=True)
        
        # Select top models based on Q-values (at least 3, at most 20)
        num_select = min(max(3, len(valid_models_list) // 2), 20)
        
        # Ensure diversity: select from all 3 groups
        selected_groups = set()
        selected_count = 0
        
        for idx, q_val in valid_q_pairs:
            if selected_count >= num_select:
                break
                
            group_idx = idx // 12
            
            # Encourage group diversity
            if selected_count < 3 or len(selected_groups) < 3:
                if group_idx not in selected_groups or selected_count < 3:
                    action[idx] = 1
                    selected_count += 1
                    selected_groups.add(group_idx)
            elif selected_count < num_select:
                action[idx] = 1
                selected_count += 1
                selected_groups.add(group_idx)
        
        return action

    def replay(self, batch_size=64):
        if len(self.memory) < batch_size:
            return None
        
        batch = random.sample(self.memory, batch_size)
        states = np.array([e[0] for e in batch])
        actions = np.array([e[1] for e in batch])
        rewards = np.array([e[2] for e in batch])
        next_states = np.array([e[3] for e in batch])
        dones = np.array([e[4] for e in batch])

        current_q_values = self.q_network.predict(states, verbose=0)
        next_q_values = self.target_network.predict(next_states, verbose=0)

        target_q_values = current_q_values.copy()
        
        for i in range(batch_size):
            # Update Q-values for all actions based on reward
            if dones[i]:
                target_value = rewards[i]
            else:
                max_next_q = np.max(next_q_values[i])
                target_value = rewards[i] + self.gamma * max_next_q
            
            # Update Q-values for selected actions
            selected_actions = np.where(actions[i] == 1)[0]
            for action_idx in selected_actions:
                target_q_values[i][action_idx] = target_value

        history = self.q_network.fit(states, target_q_values, epochs=1, verbose=0, batch_size=32)
        current_loss = history.history['loss'][0]
        self.loss_history.append(current_loss)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        
        self.epsilon_history.append(self.epsilon)
        return current_loss

    def reset_for_new_fold(self):
        """Reset agent state for new fold while keeping learned experience"""
        self.epsilon = 1.0

# =============================================================================
# ENVIRONMENT CLASS (UPDATED WITH 36-BIT STATE)
# =============================================================================
class TenFoldEnsembleEnvironment:
    def __init__(self, ensemble_evaluator, valid_models_list, fold_idx):
        self.ensemble_evaluator = ensemble_evaluator
        self.valid_models = valid_models_list
        self.fold_idx = fold_idx
        
        print(f"Environment initialized for fold {fold_idx + 1}")
        print(f"Valid models: {len(self.valid_models)}")
        print(f"State: 36-bit model selection vector")
        print(f"Action: Include/exclude decision for each model")
        self.reset()

    def reset(self):
        self.current_selection = np.zeros(36)
        return self._get_state()

    def _get_state(self):
        """State representation: 36-bit current selection"""
        return self.current_selection.copy()

    def step(self, action):
        """
        Apply action: new 36-bit selection
        Calculate reward: F1 score of new selection
        """
        self.current_selection = action.copy()
        selected_indices = np.where(action == 1)[0]
        
        valid_selected = [i for i in selected_indices if i in self.valid_models]
        
        try:
            # Calculate reward (F1 score) for this selection
            reward = self.ensemble_evaluator.evaluate_ensemble_for_dqn_training(valid_selected, self.fold_idx)
        except Exception as e:
            print(f"Error calculating ensemble reward: {e}")
            reward = 0.0
        
        return self._get_state(), reward, True
# =============================================================================
# 10-FOLD TRAINING FUNCTION (UPDATED WITH MODEL RETRAINING)
# =============================================================================
def train_10fold_ensemble_dqn(original_model_templates, combined_datasets, external_datasets, 
                               episodes=1000, evaluation_frequency=50, f1_curve_frequency=1, 
                               model_training_epochs=200):
    """Train DQN with 10-fold cross-validation - models retrained on each fold"""
    
    # Create fold datasets with proper separation
    fold_datasets = create_fold_datasets(combined_datasets, n_folds=10)
    
    # Storage for all fold results
    all_fold_best_selections = []
    all_fold_best_rewards = []
    all_fold_agents = []
    
    # Storage for plotting (aggregated data)
    all_fold_rewards = []
    all_fold_best_rewards_plot = []
    all_fold_f1_scores = []
    all_fold_sizes = []
    all_fold_validation_f1 = []
    all_fold_test_f1 = []
    all_evaluation_episodes = []
    
    # Storage for final evaluation (will use best fold's retrained models)
    best_fold_retrained_models = None
    
    print(f"\n{'='*70}")
    print(f"STARTING 10-FOLD CROSS-VALIDATION WITH MODEL RETRAINING")
    print(f"{'='*70}")
    print(f"Episodes per fold: {episodes}")
    print(f"Model retraining epochs: {model_training_epochs}")
    print(f"State representation: Only F1 score")
    
    # Train on each fold
    for fold_idx in range(10):
        # CRITICAL: Retrain all 36 models on this fold's model_training data
        retrained_models = retrain_models_for_fold(
            original_model_templates, 
            fold_datasets, 
            fold_idx,
            epochs=model_training_epochs
        )
        
        # Initialize evaluator with retrained models for this fold
        evaluator = TenFoldEnsembleEvaluator(retrained_models, combined_datasets, fold_datasets)
        valid_models_list = evaluator.valid_models
        
        print(f"\n{'='*70}")
        print(f"FOLD {fold_idx + 1}/10: DQN ENSEMBLE OPTIMIZATION")
        print(f"{'='*70}")
        
        # Initialize environment and agent for this fold
        env = TenFoldEnsembleEnvironment(evaluator, valid_models_list, fold_idx)
        
        agent = EnsembleDQNAgent(state_size=36, action_size=36)  # state_size=36
        
        # Reset agent for new fold (optional transfer learning)
        if fold_idx > 0:
            agent.reset_for_new_fold()
        
        best_selection = np.zeros(36)
        best_reward = -float('inf')
        best_episode = 0
        
        episode_rewards = []
        episode_best_rewards = []
        episode_f1_scores = []
        episode_sizes = []
        
        # F1 learning curve tracking for this fold
        episode_validation_f1 = []
        episode_test_f1 = []
        f1_evaluation_episodes = []
        
        # Train for this fold
        for episode in range(episodes):
            state = env.reset()
            
            action = agent.act(state, valid_models_list)
            next_state, reward, done = env.step(action)
            
            agent.remember(state, action, reward, next_state, done)
            
            episode_rewards.append(reward)
            agent.reward_history.append(reward)
            
            selected_count = int(np.sum(action))
            episode_sizes.append(selected_count)
            episode_f1_scores.append(reward)
            
            if reward > best_reward:
                best_reward = reward
                best_selection = action.copy()
                best_episode = episode
            
            episode_best_rewards.append(best_reward)
            
            loss = agent.replay()
            
            if episode % 25 == 0:
                agent.update_target_network()
            
            # F1 learning curve evaluation (uses independent validation/test split)
            if episode % f1_curve_frequency == 0:
                selected_indices = np.where(action == 1)[0]
                validation_f1, test_f1 = evaluator.evaluate_ensemble_for_learning_curve(selected_indices, fold_idx)
                episode_validation_f1.append(validation_f1)
                episode_test_f1.append(test_f1)
                f1_evaluation_episodes.append(episode)
            
            # Progress reporting
            if episode % evaluation_frequency == 0:
                selected_count = int(np.sum(best_selection))
                print(f"Fold {fold_idx + 1} | Episode {episode:4d} | Best F1: {best_reward:.4f} | "
                      f"Size: {selected_count:2d} | Current F1: {reward:.4f} | ε: {agent.epsilon:.3f}")
        
        # Store fold results
        all_fold_best_selections.append(best_selection)
        all_fold_best_rewards.append(best_reward)
        all_fold_agents.append(agent)
        
        # Store plotting data
        all_fold_rewards.append(episode_rewards)
        all_fold_best_rewards_plot.append(episode_best_rewards)
        all_fold_f1_scores.append(episode_f1_scores)
        all_fold_sizes.append(episode_sizes)
        all_fold_validation_f1.append(episode_validation_f1)
        all_fold_test_f1.append(episode_test_f1)
        all_evaluation_episodes.append(f1_evaluation_episodes)
        
        # Store retrained models if this is the best fold so far
        if best_reward == max(all_fold_best_rewards):
            best_fold_retrained_models = retrained_models
        
        print(f"\n✅ Fold {fold_idx + 1} completed! Best F1: {best_reward:.4f} at episode {best_episode}")
    
    # Select best ensemble based on highest performance
    best_fold_idx = np.argmax(all_fold_best_rewards)
    best_overall_selection = all_fold_best_selections[best_fold_idx]
    best_overall_reward = all_fold_best_rewards[best_fold_idx]
    
    print(f"\n{'='*70}")
    print(f"10-FOLD CROSS-VALIDATION COMPLETED")
    print(f"{'='*70}")
    print(f"Best fold: {best_fold_idx + 1}")
    print(f"Best ensemble F1 score: {best_overall_reward:.4f}")
    print(f"Average F1 across folds: {np.mean(all_fold_best_rewards):.4f} ± {np.std(all_fold_best_rewards):.4f}")
    
    # Create all required visualizations (single aggregated curves)
    print("\nCreating aggregated visualization plots...")
    
    # 1. Training reward plots with aggregated convergence
    create_aggregated_ensemble_training_plots(all_fold_rewards, all_fold_best_rewards_plot, 
                                         all_fold_f1_scores, all_fold_sizes)
    
    # 2. DQN learning plots (aggregated Loss, Epsilon, Q-Value curves)
    create_aggregated_dqn_learning_plots(all_fold_agents, all_fold_f1_scores)
    
    # 3. F1 learning curves (aggregated Validation vs Test curves)
    if len(all_fold_validation_f1[0]) > 0:
        create_aggregated_f1_learning_curve(all_fold_validation_f1, all_fold_test_f1, all_evaluation_episodes)
    
    # Create final evaluator with best fold's retrained models for external evaluation
    final_evaluator = TenFoldEnsembleEvaluator(best_fold_retrained_models, combined_datasets, fold_datasets)
    
    return best_overall_selection, best_overall_reward, all_fold_best_selections, all_fold_best_rewards, final_evaluator, external_datasets

# =============================================================================
# MAIN EXECUTION WITH 10-FOLD CV
# =============================================================================
print(f"\n{'='*70}")
print(f"10-FOLD CV DATA FLOW WITH PROPER SEPARATION")
print(f"{'='*70}")
print("1. COMBINED DATASET: Merge original train + test data")
print("2. 10-FOLD SPLITS: Stratified splits (90% train, 10% test per fold)")
print("3. TRAINING SPLIT (90%):")
print("   - Model Training (70%): Retrain 36 models on each fold")
print("   - DQN Validation (10%): Calculate DQN rewards")
print("   - LC Validation (10%): Learning curve validation")
print("4. TEST SPLIT (10%): Learning curve test")
print("5. STATE: Only F1 score (1 feature)")
print("6. FINAL EVALUATION: External data (untouched)")

# Train with 10-fold cross-validation (models retrained on each fold)
best_selection, best_reward, all_fold_selections, all_fold_best_rewards, evaluator, external_datasets = train_10fold_ensemble_dqn(
    original_model_templates, combined_datasets, external_datasets, 
    episodes=1000, evaluation_frequency=50, f1_curve_frequency=1,
    model_training_epochs=200  # Epochs for retraining models on each fold
)

# Extract selected models
selected_model_indices = np.where(best_selection == 1)[0].tolist()

print(f"\n{'='*70}")
print(f"OPTIMAL ENSEMBLE FOUND (10-FOLD CV)")
print(f"{'='*70}")
print(f"Selected Model Indices: {selected_model_indices}")
print(f"Ensemble Size: {len(selected_model_indices)}")
print(f"Best Ensemble F1 Score: {best_reward:.4f}")
print(f"Cross-validation F1 Mean: {np.mean(all_fold_best_rewards):.4f} ± {np.std(all_fold_best_rewards):.4f}")

# Show detailed model-dataset mapping
print(f"\nModel-Dataset Combinations:")
group_counts = [0, 0, 0]
for idx in selected_model_indices:
    if 0 <= idx <= 11:
        group_idx = 0
        model_in_group = idx + 1
    elif 12 <= idx <= 23:
        group_idx = 1
        model_in_group = (idx - 12) + 1
    else:
        group_idx = 2
        model_in_group = (idx - 24) + 1
    
    dataset_group = group_idx + 1
    group_counts[group_idx] += 1
    print(f"   Model {model_in_group:2d} from Dataset Group {dataset_group} (Global Index: {idx})")

print(f"\nGroup Distribution: Group1={group_counts[0]}, Group2={group_counts[1]}, Group3={group_counts[2]}")

# Final evaluation on external validation data
print(f"\n{'='*70}")
print("FINAL EVALUATION ON EXTERNAL VALIDATION DATA")
print(f"{'='*70}")

# Direct voting ensemble evaluation on external data
voting_results = evaluate_direct_voting_ensemble_on_external_data(evaluator, selected_model_indices, external_datasets)

if voting_results:
    print(f"\n✅ 10-FOLD CV DIRECT VOTING ENSEMBLE COMPLETED")
    print(f"   - Total models voting: {voting_results['total_models']}")
    print(f"   - F1 Score: {voting_results['f1_score']:.4f}")
    print(f"   - Accuracy: {voting_results['accuracy']:.4f}")
    print(f"   - ROC-AUC: {voting_results['roc_auc']:.4f}")
    print(f"   - PR-AUC: {voting_results['pr_auc']:.4f}")
    
    # Performance comparison across folds
    cv_f1 = np.mean(all_fold_best_rewards)
    cv_std = np.std(all_fold_best_rewards)
    voting_f1 = voting_results['f1_score']
    
    print(f"\nPERFORMANCE COMPARISON:")
    print(f"   10-Fold CV F1 (mean ± std):       {cv_f1:.4f} ± {cv_std:.4f}")
    print(f"   Best Fold F1:                     {best_reward:.4f}")
    print(f"   Direct Voting F1 (external):      {voting_f1:.4f}")
    
    voting_gap = abs(best_reward - voting_f1)
    print(f"   Generalization Gap:                {voting_gap:.4f}")
    
    if voting_gap < 0.05:
        print(f"   Generalization Quality: EXCELLENT")
    elif voting_gap < 0.10:
        print(f"   Generalization Quality: GOOD")
    else:
        print(f"   Generalization Quality: MODERATE")

print(f"\n{'='*70}")
print(f"FINAL SUMMARY")
print(f"{'='*70}")
print(f"Method: 10-Fold CV + DQN + Direct Voting")
print(f"State representation: F1 score only (1 feature)")
print(f"Models retrained: 10 times (once per fold)")
print(f"Ensemble size: {len(selected_model_indices)} models")
print(f"CV F1 score: {cv_f1:.4f} ± {cv_std:.4f}")
print(f"External F1 score: {voting_results['f1_score']:.4f}" if voting_results else "N/A")
print(f"External ROC-AUC: {voting_results['roc_auc']:.4f}" if voting_results else "N/A")
print(f"External PR-AUC: {voting_results['pr_auc']:.4f}" if voting_results else "N/A")

print(f"\n✅ BENEFITS:")
print("✓ Proper 10-fold cross-validation")
print("✓ Models retrained on each fold (no data leakage)")
print("✓ Simplified state: Only F1 score")
print("✓ Proper data separation (70%-10%-10%-10%)")
print("✓ Realistic performance estimates")
print("✓ Direct democratic voting")
print("✓ External validation on untouched data")